In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from keras.layers import Dense,LSTM, Concatenate, Input, Conv1D, Flatten, Dropout, BatchNormalization, GlobalAveragePooling1D, MaxPooling1D, GlobalAveragePooling1D, UpSampling1D, concatenate, Conv1DTranspose, Cropping1D, UpSampling1D, ZeroPadding1D, Reshape
from keras.models import Sequential,Model
from sklearn.model_selection import train_test_split
from generator import Generator
from keras import metrics
import importlib

In [2]:
from datasetLoader import *

In [3]:
def plot():
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_loss(history):
    plt.plot(history.history['loss'], label='loss', marker=".")
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Error [MPG]')
    plot()

In [4]:
with open("data/initial kernels/Kernel_Silica_Adsorption.npy", 'rb') as f:
    data_sorb = np.load(f)
    data_sorb_tensor = tf.constant((data_sorb.T[87:]).T)
    data_sorb_tensor = tf.cast(data_sorb_tensor, tf.float32)

def custom_loss(y_true, y_pred):
    pass
    #print("data_sorb_tensor.shape = ", data_sorb_tensor.shape, y_pred[:,:128].shape)
    isotherm_from_distribution = tf.tensordot(y_pred[:,:128], data_sorb_tensor, axes=1)
    #print("isotherm_from_distribution = ", isotherm_from_distribution.shape, y_pred[:,128:].shape)
    return 1*tf.reduce_sum(tf.square(y_pred[:,:128] - y_true[:,:128])) + 0.5*tf.reduce_sum(tf.square(y_pred[:,128:] - isotherm_from_distribution))

# np.ediff1d(pore_widths, to_begin=pore_widths[0])

In [114]:
def create_model(x):
    layer = []
    layer.append(Input(shape=x[0].shape))
    layer.append(Dense(200, activation='relu')(layer[-1]))
    layer.append(Dense(200, activation='relu')(layer[-1]))
    layer.append(Dense(200, activation='relu')(layer[-1]))
    layer.append(Dense(200, activation='relu')(layer[-1]))
    layer.append(Dense(128, activation='relu')(layer[-1]))
    layer.append(Dense(128, activation='relu')(layer[-1]))
    layer.append(Dense(128, activation='relu')(layer[-1]))
    layer.append(Dense(128, activation='relu')(layer[-1]))
    layer.append(Concatenate()([layer[-1], layer[0]]))
    model = tf.keras.Model(inputs=layer[0], outputs=layer[-1])
    model.compile(loss=custom_loss, optimizer='Adam')
    #model.compile(loss='mean_squared_error', optimizer='Adam')
    return model

def create_model_LSTM(x):
    model = Sequential()
    model.add(LSTM(units=64, input_shape=(371, 1), return_sequences=False, activation = "elu"))
    #model.add(LSTM(units=64, activation = "elu"))
    model.add(Dense(units=128, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='Adam')
    return model

def create_model_CNN(x):
    inputs = Input(shape=(371, 1), name="Input")
    c1 = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same', name="c1")(inputs)
    p1 = MaxPooling1D(pool_size=2, padding='same', name="p1")(c1)

    c2 = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same', name="c2")(p1)
    p2 = MaxPooling1D(pool_size=2, padding='same', name="p2")(c2)
    
    c3 = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', name="c3")(p2)
    p3 = MaxPooling1D(pool_size=2, padding='same', name="p3")(c3)
    
    c4 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', name="c4")(p3)
    p4 = MaxPooling1D(pool_size=2, padding='same', name="p4")(c4)
    
    c5 = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same', name="c5")(p4)
    p5 = MaxPooling1D(pool_size=2, padding='same', name="p5")(c5)
    
    f1 = Flatten(name="f1")(p5)
    
    d1 = Dense(128, activation='relu', name="d1")(f1)
    dp1 = Dropout(0.1)(d1)
    d2 = Dense(128, activation='relu', name="d2")(dp1)
    dp2 = Dropout(0.1)(d2)
    d3 = Dense(128, activation='relu', name="d3")(dp2)
    
    input_for_loss = Reshape((371, ))(inputs)
    outputs = Concatenate()([d3, input_for_loss])
    
    #outputs = d3
    
    model = Model(inputs=[inputs], outputs=[outputs], name="CNN")
    #model.compile(loss='mean_squared_error', optimizer='Adam')
    model.compile(loss=custom_loss, optimizer='Adam')
    return model

In [ ]:
# with open("data/datasets/reports_best_tyhanov.npz", 'rb') as f:
#     dataset = np.load(f)
#     isotherm_data = dataset["isotherm_data"]
#     pore_distribution_data = dataset["pore_distribution_data"]
#     pore_widths = np.load("data/initial kernels/Size_Kernel_Silica_Adsorption.npy")

In [107]:
#x, y = load_dataset('data/datasets/reports_best_tyhanov.npz')
x, y = load_dataset('data/datasets/silica_random.npz')

In [108]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=1)

In [68]:
with open(f'data/datasets/tyhanov_test.npz', "wb") as f:
    np.savez_compressed(f, x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

In [118]:
model = create_model_CNN(x)

In [116]:
model.summary()

Model: "CNN"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, 371, 1)]     0           []                               
                                                                                                  
 c1 (Conv1D)                    (None, 371, 8)       32          ['Input[0][0]']                  
                                                                                                  
 p1 (MaxPooling1D)              (None, 186, 8)       0           ['c1[0][0]']                     
                                                                                                  
 c2 (Conv1D)                    (None, 186, 16)      400         ['p1[0][0]']                     
                                                                                                

In [99]:
for layer in model.layers:
    layer.trainable = False
for i in range(11, 15):
    model.layers[i].trainable = True
for layer in model.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x000001EDEA2BE4A0> False
<keras.layers.convolutional.conv1d.Conv1D object at 0x000001EDEA2BCD00> False
<keras.layers.pooling.max_pooling1d.MaxPooling1D object at 0x000001EDEA2BEFE0> False
<keras.layers.convolutional.conv1d.Conv1D object at 0x000001EDE98D3F40> False
<keras.layers.pooling.max_pooling1d.MaxPooling1D object at 0x000001EDE98D2440> False
<keras.layers.convolutional.conv1d.Conv1D object at 0x000001EDE9638730> False
<keras.layers.pooling.max_pooling1d.MaxPooling1D object at 0x000001EDE96385E0> False
<keras.layers.convolutional.conv1d.Conv1D object at 0x000001EE00075BD0> False
<keras.layers.pooling.max_pooling1d.MaxPooling1D object at 0x000001EDEA2BD480> False
<keras.layers.convolutional.conv1d.Conv1D object at 0x000001EE29C75030> False
<keras.layers.pooling.max_pooling1d.MaxPooling1D object at 0x000001EE29C747F0> False
<keras.layers.reshaping.flatten.Flatten object at 0x000001EE29C75F00> True
<keras.layers.core.dense.Dense object

In [123]:
from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.001)
mcp_save = tf.keras.callbacks.ModelCheckpoint(filepath='data/models/silica_CNN4_PINN.keras', save_best_only=True,
                                              monitor='val_loss', mode='min', verbose=1, save_weights_only=False,
                                              save_freq='epoch')

reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                                      patience=50, verbose=1, mode='auto')
history = model.fit(np.array(x_train), np.array(y_train),
                    epochs=50, batch_size=512, shuffle=True,
                    validation_data=(np.array(x_train), np.array(y_train)), callbacks=[reduce_lr_loss, mcp_save])
plot_loss(history)

Epoch 1/50
167/167 [==============================] - ETA: 0s - loss: 48.6693
Epoch 1: val_loss improved from inf to 42.67891, saving model to data/models\silica_CNN4_PINN.keras
167/167 [==============================] - 88s 27ms/step - loss: 48.6693 - val_loss: 42.6789 - lr: 0.0010
Epoch 2/50
166/167 [============================>.] - ETA: 0s - loss: 47.8720
Epoch 2: val_loss did not improve from 42.67891
167/167 [==============================] - 4s 25ms/step - loss: 47.8675 - val_loss: 48.2501 - lr: 0.0010
Epoch 3/50
165/167 [============================>.] - ETA: 0s - loss: 47.7449
Epoch 3: val_loss improved from 42.67891 to 40.31203, saving model to data/models\silica_CNN4_PINN.keras
167/167 [==============================] - 5s 31ms/step - loss: 47.7462 - val_loss: 40.3120 - lr: 0.0010
Epoch 4/50
167/167 [==============================] - ETA: 0s - loss: 47.5483
Epoch 4: val_loss improved from 40.31203 to 36.19619, saving model to data/models\silica_CNN4_PINN.keras
167/167 [=====

In [46]:
plot_loss(history)

In [120]:
# load model
model = tf.keras.models.load_model('data/models/silica_CNN4_PINN.keras', custom_objects={'abs': tf.math.abs, 'custom_loss': custom_loss})

In [83]:
#tf.keras.models.save_model(model,'data/models/silica_CNN3.keras')

In [121]:
# prediction = model.evaluate(np.array(x_test), np.array(y_test))
prediction = model.predict(np.array(x_test))

469/469 [==============================] - 8s 10ms/step


In [122]:
# test on x_train
def fetch_prediction(prediction):
    return prediction[:128]

pore_widths = np.load("data/initial kernels/Size_Kernel_Silica_Adsorption.npy")
pressures = np.load("data/initial kernels/Pressure_Silica.npy")
NX, NY = 3, 4
figure, axis = plt.subplots(NX, NY)
for i in range(NX):
    for j in range(NY):
        k = np.random.randint(0, len(x_test))
        x_scale_factor = max(pore_widths) / len(x_test[k])
        axis[i, j].plot(pore_widths / x_scale_factor, fetch_prediction(prediction[k]), marker=".", label=f"Prediction")
        axis[i, j].plot(pore_widths / x_scale_factor, y_test[k], marker=".", label="Real distribution")
        axis[i, j].plot(pressures[77:-10]*500, x_test[k], label="Isotherm")
        kernal = (data_sorb.T[40:])
        axis[i, j].plot(pressures[40:]*500, np.dot(kernal, prediction[k][:128]), label="New Isotherm")
        axis[i, j].set_title(f"№ {k}")
        axis[i, j].title.set_size(10)
plt.subplots_adjust(hspace=0.6, right=0.95, left=0.05, bottom=0.05, top=0.95)
plt.legend()
plot()

In [ ]:
# test with test Generator
from tools import TestApp

gen = Generator(path_s="data/initial kernels/Kernel_Silica_Adsorption.npy",
                path_d="data/initial kernels/Kernel_Silica_Desorption.npy",
                path_p_d="data/initial kernels/Pressure_Silica.npy",
                path_p_s="data/initial kernels/Pressure_Silica.npy",
                path_a="data/initial kernels/Size_Kernel_Silica_Adsorption.npy"
                )
# gen = Generator(path_s="data/initial kernels/Kernel_Carbon_Adsorption.npy",
#                               path_d="data/initial kernels/Kernel_Carbon_Desorption.npy",
#                               path_p_d="data/initial kernels/Pressure_Carbon.npy",
#                               path_p_s="data/initial kernels/Pressure_Carbon.npy",
#                               path_a="data/initial kernels/Size_Kernel_Carbon_Adsorption.npy"
#                             )

#TestApp.App(model, gen)

In [ ]:
exp_file_list = ["MCM-41", "SBA-15", "SBA-16", "MIL-101", "MIL-101_2", "DUT-49", "FDM-4", "PCN-333", "PCN-777",
                 "MIL-100"]

p_exp_list = []
n_s_exp_raw_list = []
for exp_file_name in exp_file_list:
    data = pd.read_csv(f"data/real/{exp_file_name}.txt", header=None)
    # p_exp_list.append(data.iloc[:,1].to_numpy())
    # n_s_exp_raw_list.append(data.iloc[:,3].to_numpy())
    p_exp_list.append(data.iloc[:, 1].to_numpy())
    n_s_exp_raw_list.append(data.iloc[:, 3].to_numpy())

In [ ]:
j = 2
plt.plot(p_exp_list[j], n_s_exp_raw_list[j], marker=".", label=exp_file_list[j])
plot()

In [ ]:
# интерполируем экспериментальную изотерму под давления кернала
n_s_exp_list = []
for i in range(len(p_exp_list)):
    n_s_exp_list.append(np.interp(gen.pressures_s[77:367], p_exp_list[i], n_s_exp_raw_list[i]))

In [ ]:
j = 2
plt.plot(gen.pressures_s[77:367], n_s_exp_list[j], marker=".", label=exp_file_list[j])
plot()

In [ ]:
n_s_exp_for_net_list = [pre_process_isotherm(np.copy(n_s_exp), scale=False) for n_s_exp in n_s_exp_list]
fit_exp_list = [model.predict(np.array([n_s_exp_for_net])).T for n_s_exp_for_net in n_s_exp_for_net_list]
fit_exp_list = [fetch_prediction(i) for i in fit_exp_list]

In [ ]:
NX, NY = 3, 4
figure, axis = plt.subplots(NX, NY)
k = 0
for i in range(NX):
    for j in range(NY):
        x_scale_factor = max(gen.a_array) / max(p_exp_list[k])
        y_scale_factor = max(fit_exp_list[k]) / max(n_s_exp_raw_list[k])
        axis[i, j].plot(pore_widths, fit_exp_list[k], marker=".", label=f"Distribution")
        axis[i, j].plot(p_exp_list[k] * x_scale_factor, n_s_exp_raw_list[k],
                        label=f"{exp_file_list[k]}", marker=".")
        
        kernal = (data_sorb.T[40:])
        axis[i, j].plot(gen.pressures_s[40:458]* x_scale_factor, np.dot(kernal, fit_exp_list[k]), label="Isotherm by distribution")
        
        axis[i, j].set_title(f"max at {round(gen.a_array[np.argmax(fit_exp_list[k])], 2)} nm")
        axis[i, j].title.set_size(12)
        axis[i, j].legend(loc="upper right")
        axis[i, j].grid()
        k += 1
        if k >= len(fit_exp_list):
            break
plt.subplots_adjust(hspace=0.6, right=0.95, left=0.05, bottom=0.05, top=0.95)
plot()

In [ ]:
### Classic
import inverse

kernel = np.load("data/initial kernels/Kernel_Silica_Adsorption.npy")

### normalize on size
# a_array = np.load("data/initial kernels/Size_Kernel_Silica_Adsorption.npy")
# for i in range(len(a_array)):
#     kernel[i] /= a_array[i]
#     kernel[i] /= a_array[i]
###

cut_kernel = []
for i in range(len(kernel)):
    cut_kernel.append(kernel[i][40:458])
cut_kernel = np.array(cut_kernel)
fit_classic_list = [inverse.fit_SLSQP(adsorption=n_s, kernel=cut_kernel, a_array=pore_widths) for n_s in n_s_exp_list]

In [ ]:
from matplotlib import animation

importlib.reload(inverse)


def create_regularization_animation(file):
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 5, True)
    fit_classic = inverse.fit_SLSQP(adsorption=n_s_exp_list[2], kernel=cut_kernel, a_array=pore_widths, alpha=0)
    line1, = ax.plot(pore_widths[:-30], fit_classic.x[:-30], marker=".", label=f"a = {0}")

    y_scale_factor = max(fit_classic.x) / max(fit_exp_list[2])
    #plt.plot(pore_widths, fit_exp_list[2] * y_scale_factor, marker=".", label=f"Суррогатная модель")

    ax.set_ylabel("Объем пор, $см^3$/ нм * г")
    ax.set_xlabel("Размер пор, нм")

    L = plt.legend(loc=1)  # Define legend objects

    def update(frame):
        a = frame / 4 + 2
        fit_classic = inverse.fit_SLSQP(adsorption=n_s_exp_list[2], kernel=cut_kernel, a_array=pore_widths, alpha=a)
        line1.set_ydata(fit_classic.x[:-30])
        L.get_texts()[0].set_text(
            f"Распределение пор по размерам, параметр регуляризации α = {round(a-2,1)}")  # Update label each at frame
        return line1,

    ani = animation.FuncAnimation(fig=fig, func=update, frames=100, interval=100)
    writervideo = animation.FFMpegWriter(fps=30)
    ani.save(file, writer=writervideo, dpi=200)
    plt.grid()
    plt.show()
create_regularization_animation("SBA-16_regularization.mp4")
def plot_regularization_graphs():
    for a in [1, 5, 10, 50]:
        fit_classic = inverse.fit_SLSQP(adsorption=n_s_exp_list[2], kernel=cut_kernel, a_array=pore_widths, alpha=a)
        plt.plot(pore_widths, fit_classic.x, marker=".", label=f"α = {a}")
    plt.ylabel("Объем пор, $см^3$/ нм * гр")
    plt.xlabel("Размер пор, нм")
    plot()
#plot_regularization_graphs()

In [ ]:
NX, NY = 3, 4
figure, axis = plt.subplots(NX, NY)
k = 0
for i in range(NX):
    for j in range(NY):
        x_scale_factor = max(gen.a_array) / max(p_exp_list[k])
        y_scale_factor = max(fit_classic_list[k].x) / max(n_s_exp_raw_list[k])
        y_scale_factor_net = max(fit_classic_list[k].x) / max(fit_exp_list[k])

        axis[i, j].plot(pore_widths, fit_exp_list[k] * y_scale_factor_net, marker=".", label=f"net")
        axis[i, j].plot(pore_widths, fit_classic_list[k].x, marker=".", label=f"classic")
        axis[i, j].plot(p_exp_list[k] * x_scale_factor, n_s_exp_raw_list[k] * y_scale_factor,
                        label=f"{exp_file_list[k]}", marker=".")
        axis[i, j].set_title(f"max at {round(gen.a_array[np.argmax(fit_classic_list[k].x)], 2)} nm")
        axis[i, j].title.set_size(12)
        axis[i, j].legend(loc="upper right")
        axis[i, j].grid()
        k += 1
        if k >= len(fit_exp_list):
            break
plt.subplots_adjust(hspace=0.6, right=0.95, left=0.05, bottom=0.05, top=0.95)
plot()

In [ ]:
### Classic
import inverse

kernel = np.load("data/initial kernels/Kernel_Silica_Adsorption.npy")

### normalize on size
# a_array = np.load("data/initial kernels/Size_Kernel_Silica_Adsorption.npy")
# for i in range(len(a_array)):
#     kernel[i] /= a_array[i]
#     kernel[i] /= a_array[i]
###

cut_kernel = []
for i in range(len(kernel)):
    cut_kernel.append(kernel[i][40:458])
cut_kernel = np.array(cut_kernel)
fit_classic_list = [inverse.fit_SLSQP(adsorption=n_s, kernel=cut_kernel, a_array=pore_widths) for n_s in n_s_exp_list]

In [ ]:
from matplotlib import animation

importlib.reload(inverse)


def create_regularization_animation(file):
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 5, True)
    fit_classic = inverse.fit_SLSQP(adsorption=n_s_exp_list[2], kernel=cut_kernel, a_array=pore_widths, alpha=0)
    line1, = ax.plot(pore_widths[:-30], fit_classic.x[:-30], marker=".", label=f"a = {0}")

    y_scale_factor = max(fit_classic.x) / max(fit_exp_list[2])
    #plt.plot(pore_widths, fit_exp_list[2] * y_scale_factor, marker=".", label=f"Суррогатная модель")

    ax.set_ylabel("Объем пор, $см^3$/ нм * г")
    ax.set_xlabel("Размер пор, нм")

    L = plt.legend(loc=1)  # Define legend objects

    def update(frame):
        a = frame / 4 + 2
        fit_classic = inverse.fit_SLSQP(adsorption=n_s_exp_list[2], kernel=cut_kernel, a_array=pore_widths, alpha=a)
        line1.set_ydata(fit_classic.x[:-30])
        L.get_texts()[0].set_text(
            f"Распределение пор по размерам, параметр регуляризации α = {round(a-2,1)}")  # Update label each at frame
        return line1,

    ani = animation.FuncAnimation(fig=fig, func=update, frames=100, interval=100)
    writervideo = animation.FFMpegWriter(fps=30)
    ani.save(file, writer=writervideo, dpi=200)
    plt.grid()
    plt.show()
create_regularization_animation("SBA-16_regularization.mp4")
def plot_regularization_graphs():
    for a in [1, 5, 10, 50]:
        fit_classic = inverse.fit_SLSQP(adsorption=n_s_exp_list[2], kernel=cut_kernel, a_array=pore_widths, alpha=a)
        plt.plot(pore_widths, fit_classic.x, marker=".", label=f"α = {a}")
    plt.ylabel("Объем пор, $см^3$/ нм * гр")
    plt.xlabel("Размер пор, нм")
    plot()
#plot_regularization_graphs()

In [ ]:
NX, NY = 3, 4
figure, axis = plt.subplots(NX, NY)
k = 0
for i in range(NX):
    for j in range(NY):
        x_scale_factor = max(gen.a_array) / max(p_exp_list[k])
        y_scale_factor = max(fit_classic_list[k].x) / max(n_s_exp_raw_list[k])
        y_scale_factor_net = max(fit_classic_list[k].x) / max(fit_exp_list[k])

        axis[i, j].plot(pore_widths, fit_exp_list[k] * y_scale_factor_net, marker=".", label=f"net")
        axis[i, j].plot(pore_widths, fit_classic_list[k].x, marker=".", label=f"classic")
        axis[i, j].plot(p_exp_list[k] * x_scale_factor, n_s_exp_raw_list[k] * y_scale_factor,
                        label=f"{exp_file_list[k]}", marker=".")
        axis[i, j].set_title(f"max at {round(gen.a_array[np.argmax(fit_classic_list[k].x)], 2)} nm")
        axis[i, j].title.set_size(12)
        axis[i, j].legend(loc="upper right")
        axis[i, j].grid()
        k += 1
        if k >= len(fit_exp_list):
            break
plt.subplots_adjust(hspace=0.6, right=0.95, left=0.05, bottom=0.05, top=0.95)
plot()

In [ ]:
# compare net, classic, quantachrome distributions
NX, NY = 3, 4
figure, axis = plt.subplots(NX, NY)
k = 0


def calculate_isotherm_by_distribution(generator: Generator, a_array, distribution):
    generator.a_array = a_array
    generator.pore_distribution = distribution
    generator.calculate_isotherms()
    return generator.n_s


#/np.ediff1d(pore_widths, to_begin=pore_widths[0])
for i in range(NX):
    for j in range(NY):
        quantachrome_pore_size = \
            np.genfromtxt(f"data/real/quantachrome/silica/distribution/{exp_file_list[k]}.csv", delimiter=",")[1:].T[
                0] / 10 * 2  # /10 - перевод в НМ * 2 - в QH размер - Half pore width.
        quantachrome_dV = \
            np.genfromtxt(f"data/real/quantachrome/silica/distribution/{exp_file_list[k]}.csv", delimiter=",")[1:].T[3]
        # isotherm_formQC = calculate_isotherm_by_distribution(gen, pore_widths, np.interp(pore_widths, quantachrome_pore_size, quantachrome_dV))
        # y_scale_factor_QH = max(n_s_exp_raw_list[k]) / max(isotherm_formQC)
        # quantachrome_dV *= y_scale_factor_QH

        y_scale_factor_classic = max(quantachrome_dV) / max(fit_classic_list[k].x)
        y_scale_factor_net = max(quantachrome_dV) / max(fit_exp_list[k])

        axis[i, j].plot(pore_widths, fit_exp_list[k] * y_scale_factor_net, marker=".", label=f"net")
        axis[i, j].plot(pore_widths, fit_classic_list[k].x * y_scale_factor_classic, marker=".", label=f"classic")
        axis[i, j].plot(quantachrome_pore_size, quantachrome_dV, marker=".", label=f"quantachrome")
        axis[i, j].set_title(f"{exp_file_list[k]}")
        axis[i, j].title.set_size(12)
        axis[i, j].legend(loc="upper right")
        axis[i, j].grid()
        k += 1
        if k >= len(fit_exp_list):
            break
plt.subplots_adjust(hspace=0.6, right=0.95, left=0.05, bottom=0.05, top=0.95)
plot()

In [ ]:
# plots for presentation
import matplotlib.pyplot as plt
from importlib import reload

plt = reload(plt)

k = 0

quantachrome_pore_size = \
    np.genfromtxt(f"data/real/quantachrome/silica/distribution/{exp_file_list[k]}.csv", delimiter=",")[1:].T[
        0] / 10 * 2  # /10 - перевод в НМ * 2 - в QH размер - Half pore width.
quantachrome_dV = \
    np.genfromtxt(f"data/real/quantachrome/silica/distribution/{exp_file_list[k]}.csv", delimiter=",")[1:].T[3]
# isotherm_formQC = calculate_isotherm_by_distribution(gen, pore_widths, np.interp(pore_widths, quantachrome_pore_size, quantachrome_dV))
# y_scale_factor_QH = max(n_s_exp_raw_list[k]) / max(isotherm_formQC)
# quantachrome_dV *= y_scale_factor_QH

y_scale_factor_classic = max(quantachrome_dV) / max(fit_classic_list[k].x)
y_scale_factor_net = max(quantachrome_dV) / max(fit_exp_list[k])

plt.plot(pore_widths, fit_exp_list[k] * y_scale_factor_net, marker=".", label=f"Суррогатная модель")
plt.plot(pore_widths, fit_classic_list[k].x * y_scale_factor_classic, marker=".", label=f"Математическое решение")
#plt.plot(quantachrome_pore_size, quantachrome_dV, marker=".", label=f"Математическое решение QH") 
plt.legend(loc="upper right")
plt.grid()
plt.title(f"{exp_file_list[k]}")

# plt.xscale('log')
plt.subplots_adjust(left=0.15,
                    bottom=0.133,
                    right=0.979,
                    top=0.917,
                    wspace=0.4,
                    hspace=0.4)
plt.legend()
plt.grid(True)
plt.ylabel("Объем пор, $см^3$/ нм * гр")
plt.xlabel("Размер пор, нм")
plt.show()


In [ ]:
# plots for presentation 2
k = 3
plt.plot(p_exp_list[k], n_s_exp_raw_list[k], marker=".", color='b', label=f"{exp_file_list[k]}")
k = 9
plt.plot(p_exp_list[k], n_s_exp_raw_list[k], marker=".", color='r', label=f"{exp_file_list[k]}")
plt.legend(loc="upper right")
plt.grid()
plt.title(f"Изотермы адсорбции")

# plt.xscale('log')
plt.subplots_adjust(left=0.15,
                    bottom=0.133,
                    right=0.979,
                    top=0.917,
                    wspace=0.4,
                    hspace=0.4)
plt.legend()
plt.grid(True)
plt.ylabel("Адсорбция, $см^3$/г")
plt.xlabel("Давление, $P/P_{0}$")
plt.show()


In [ ]:
# plots for presentation 3

fig, ax1 = plt.subplots(figsize=(6, 7))
ax1.set_xlabel("Размер пор, нм")
ax1.set_ylabel("Объем пор, $см^3$/ нм * гр")
k = 0
plt.title(f"{exp_file_list[k]}")
quantachrome_pore_size = \
    np.genfromtxt(f"data/real/quantachrome/silica/distribution/{exp_file_list[k]}.csv", delimiter=",")[1:].T[
        0] / 10 * 2  # /10 - перевод в НМ * 2 - в QH размер - Half pore width.
quantachrome_dV = \
    np.genfromtxt(f"data/real/quantachrome/silica/distribution/{exp_file_list[k]}.csv", delimiter=",")[1:].T[3]
# isotherm_formQC = calculate_isotherm_by_distribution(gen, pore_widths, np.interp(pore_widths, quantachrome_pore_size, quantachrome_dV))
# y_scale_factor_QH = max(n_s_exp_raw_list[k]) / max(isotherm_formQC)
# quantachrome_dV *= y_scale_factor_QH

y_scale_factor_classic = max(quantachrome_dV) / max(fit_classic_list[k].x)
y_scale_factor_net = max(quantachrome_dV) / max(fit_exp_list[k])

ax1.plot(pore_widths[0:100], (fit_exp_list[k] * y_scale_factor_net)[0:100], marker=".", label=f"Суррогатная модель")
ax1.plot(pore_widths[0:100], (fit_classic_list[k].x * y_scale_factor_classic)[0:100], marker=".",
         label=f"Математическое решение")
# ax1.tick_params(axis='y')
# plt.legend(loc="right")


ax2 = ax1.twinx()
ax3 = ax2.twiny()  # instantiate a second axes that shares the same x-axis
ax3.set_xlabel("Давление, $P/P_{0}$")
ax2.set_ylabel("Адсорбция, $см^3$/г")  # we already handled the x-label with ax1
ax3.plot(p_exp_list[k], n_s_exp_raw_list[k], color='g', label=f"Изотерма адсорбции")
# plt.legend(loc="right")
# 
# ax2.tick_params(axis='y')
# plt.subplots_adjust(left=0.15,
#                     bottom=0.133, 
#                     right=0.979, 
#                     top=0.917, 
#                     wspace=0.4, 
#                     hspace=0.4)

fig.tight_layout()
plt.show()

In [ ]:
# Compare isotherms
NX, NY = 2, 3
figure, axis = plt.subplots(NX, NY)
k = 0

for i in range(NX):
    for j in range(NY):
        net_isotherm = calculate_isotherm_by_distribution(gen, pore_widths, fit_exp_list[k].T[0])
        classic_isotherm = calculate_isotherm_by_distribution(gen, pore_widths, fit_classic_list[k].x)
        quantachrome_data = np.genfromtxt(f"data/real/quantachrome/silica/fitting/{exp_file_list[k]}.csv",
                                          delimiter=",")[1:].T
        quantachrome_pore_size = \
            np.genfromtxt(f"data/real/quantachrome/silica/distribution/{exp_file_list[k]}.csv", delimiter=",")[1:].T[
                0] / 10 * 2
        quantachrome_dV = \
            np.genfromtxt(f"data/real/quantachrome/silica/distribution/{exp_file_list[k]}.csv", delimiter=",")[1:].T[3]
        quantachrome_data2 = calculate_isotherm_by_distribution(gen, pore_widths,
                                                                np.interp(pore_widths, quantachrome_pore_size,
                                                                          quantachrome_dV))

        y_scale_factor_net = max(net_isotherm) / max(n_s_exp_raw_list[k])
        y_scale_factor_classic = max(classic_isotherm) / max(n_s_exp_raw_list[k])
        y_scale_factor_quantachrome = max(quantachrome_data[1]) / max(n_s_exp_raw_list[k])
        y_scale_factor_quantachrome2 = max(quantachrome_data2) / (
            n_s_exp_raw_list[k][-1])  #max(quantachrome_data[1]) / max(n_s_exp_raw_list[k])
        print(1 / y_scale_factor_quantachrome2)

        axis[i, j].plot(p_exp_list[k], n_s_exp_raw_list[k], label="real")
        axis[i, j].plot(gen.pressures_s, classic_isotherm / y_scale_factor_classic, marker=".", label=f"classic")
        axis[i, j].plot(gen.pressures_s, net_isotherm / y_scale_factor_net, label="net")
        # axis[i, j].plot(quantachrome_data[0], quantachrome_data[1]/y_scale_factor_quantachrome, label="quantachrome")
        # axis[i, j].plot(gen.pressures_s, quantachrome_data2/y_scale_factor_quantachrome2, label="quantachrome_from_kernal")
        axis[i, j].set_title(f"{exp_file_list[k]}")
        axis[i, j].legend(loc="lower right")
        axis[i, j].grid()
        k += 1
        if k >= len(fit_exp_list):
            break
plt.subplots_adjust(hspace=0.6, right=0.95, left=0.05, bottom=0.05, top=0.95)
plot()

In [ ]:
#QUANTACHROME
data = np.genfromtxt("data/real/quantachrome/silica/distribution/MIL-101.csv", delimiter=",")[1:].T[0]